In [4]:
from pathlib import Path
import pickle

In [5]:
savepath = Path().absolute() / 'slurm_bashscripts'
savepath.mkdir(exist_ok=True)

In [6]:
def make_slurm_job(name, h_max, n, var, cvar, alpha, tree, year, hours):
    w = open(name, 'w', newline='\n')
    w.write('#!/bin/bash\n')
    w.write('#SBATCH -J optimization_run\n')
    w.write('#SBATCH --nodes=1\n')
    w.write('#SBATCH --ntasks=1\n')
    n_cpu = 32
    w.write(f'#SBATCH --cpus-per-task={n_cpu}\n') # can go to 128?
    # make the timestring from the hours
    timestring = str(int(hours)) + ':' + str(int((hours - int(hours))*60)) + ':00'
    w.write('#SBATCH --time=' + timestring + '\n')
    w.write('#SBATCH --partition=genoa\n')
    w.write('#SBATCH --output=slurm_%j.log\n')
    w.write('#SBATCH --error=slurm_%j.log\n')
    w.write('echo starting job\n')
    w.write("")
    w.write('module load 2022\n')
    w.write('module load Miniconda3/4.12.0\n')
    w.write('source /gpfs/admin/_hpc/sw/arch/AMD-ZEN2/RHEL8/EB_production/2022/software/Miniconda3/4.12.0/etc/profile.d/conda.sh\n')
    w.write('conda activate optimization\n')
    w.write("")
    w.write("method='energy'\n")
    w.write(f"var_wl={var}\n")
    w.write(f'max_threads={n_cpu}\n')
    w.write('start_from_scratch=False\n')
    w.write(f"p_chance=1\n")
    w.write("")
    w.write(f"wl_constraint_type='cvar'\n")
    w.write('n_scenarios=' + str(n) + '\n')
    w.write(f"cvar_wl={cvar}\n")
    w.write(f"cvar_alpha={alpha}\n")
    w.write(f'tree_based={tree}\n')
    w.write(f'year={year}\n')
    w.write(f'h_max={h_max}\n')

    w.write(f'export OMP_NUM_THREADS=$max_threads\n')
    w.write('set -euo pipefail\n')

    w.write("python 'optimization/src/run_sim_date.py' n_scenarios=$n_scenarios distance_metric=$method wl_constraint_type=$wl_constraint_type var_wl=$var_wl cvar_wl=$cvar_wl cvar_alpha=$cvar_alpha p_chance=$p_chance tree_based=$tree_based max_threads=$max_threads start_from_scratch=$start_from_scratch h_max=$h_max year=$year")


In [6]:
def make_slurm_job_robust(name, h_max, n, var, cvar, alpha, tree, year, hours):
    w = open(name, 'w', newline='\n')
    w.write('#!/bin/bash\n')
    w.write('#SBATCH -J optimization_run\n')
    w.write('#SBATCH --nodes=1\n')
    w.write('#SBATCH --ntasks=1\n')
    n_cpu = 32
    w.write(f'#SBATCH --cpus-per-task={n_cpu}\n') # can go to 128?
    # make the timestring from the hours
    timestring = str(int(hours)) + ':' + str(int((hours - int(hours))*60)) + ':00'
    w.write('#SBATCH --time=' + timestring + '\n')
    w.write('#SBATCH --partition=genoa\n')
    w.write('#SBATCH --output=slurm_%j.log\n')
    w.write('#SBATCH --error=slurm_%j.log\n')
    w.write("")
    w.write('module load 2022\n')
    w.write('module load Miniconda3/4.12.0\n')
    w.write('source /gpfs/admin/_hpc/sw/arch/AMD-ZEN2/RHEL8/EB_production/2022/software/Miniconda3/4.12.0/etc/profile.d/conda.sh\n')
    w.write('conda activate optimization\n')
    w.write("")
    w.write("method='energy'\n")
    w.write(f"var_wl={var}\n")
    w.write(f'max_threads={n_cpu}\n')
    w.write('start_from_scratch=False\n')
    w.write(f"p_chance=1\n")
    w.write("")
    w.write(f"wl_constraint_type='robust'\n")
    w.write('n_scenarios=' + str(n) + '\n')
    w.write(f"cvar_wl={cvar}\n")
    w.write(f"cvar_alpha={alpha}\n")
    w.write(f'tree_based={tree}\n')
    w.write(f'year={year}\n')
    w.write(f'h_max={h_max}\n')

    w.write(f'export OMP_NUM_THREADS=$max_threads\n')
    w.write('set -euo pipefail\n')

    w.write("python 'optimization/src/run_sim_date.py' n_scenarios=$n_scenarios distance_metric=$method wl_constraint_type=$wl_constraint_type var_wl=$var_wl cvar_wl=$cvar_wl cvar_alpha=$cvar_alpha p_chance=$p_chance tree_based=$tree_based max_threads=$max_threads start_from_scratch=$start_from_scratch h_max=$h_max year=$year")


In [3]:
# 16/09/23 1043:  finish scen 3's, again
basename = 'optrun_dates'
alphas = [0.9, 0.9, 0.9, 0.8, 0.9, 0.9, 0.9, 0.8, 0.8, 0.8, 0.9, 0.9, 0.8, 0.8, 0.8]
cvars = [-0.3] * 15# + [-0.395] + [-0.3] * 6 + [-0.395] + [-0.3] * 3
years = [2019, 2020, 2021, 2020, 2019, 2020, 2021, 2019, 2020, 2021, 2019, 2020, 2019, 2020, 2021]
trees = [False] * 10 + [True] * 5
n_scenarios = [3] * 15
h_maxs = [-0.4] * 4 + [-0.3] * 11
hours = [24 if y!=2021 else 8 for y in years]

assert len(alphas) == len(cvars) == len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [10]:
# 09/10/23 2036 (NL)
basename = 'optrun_dates_'
alphas = [0.99] * 3 + [0.9] * 6 + [0.8] * 6 + [0.3] * 2 + [0.8] * 2 + [0.99] * 3 + [0.9] * 6 + [0.8] * 6 + [0.9] * 2 + [0.8] * 2
cvars = [-0.3] * 3 + [-0.395] * 3 + [-0.3] * 3 + [-0.395] * 3 + [-0.3] * 3 + [-0.3] * 4 + [-0.3] * 3 + [-0.395] * 3 + [-0.3] * 3 + [-0.395] * 3 + [-0.3] * 3 + [-0.3] * 4
years = [2019, 2020, 2021] * 5 + [2019, 2020] * 2 + [2019, 2020, 2021] * 5 + [2019, 2020] * 2
trees = [False] * 19 + [False] * 15 + [True] * 4
n_scenarios = [5] * 15 + [3] * 4 + [5] * 15 + [5] * 4
h_maxs = [-0.4] * 15 + [-0.3] * 4 + [-0.3] * 15 + [-0.3] * 4
hours = [48 if y!=2021 else 24 for y in years]
assert len(alphas) == len(cvars) == len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [14]:
# 05/11/23 1020
basename = 'optrun_dates_'
alphas = [0.99] * 2 + [0.9] * 4 + [0.8] * 2 + [0.99] * 2 + [0.9] * 5 + [0.8] * 5
cvars = [-0.3] * 2 + [-0.395] * 1 + [-0.3] * 3 + [-0.395] * 2 + [-0.3] * 2 + [-0.395] * 1 + [-0.3] * 4 + [-0.395] * 2 + [-0.3] * 3
years = [2019, 2020] + [2019, 2019, 2020, 2021] + [2019, 2020] + [2019, 2020] + [2019] + [2019, 2019, 2020, 2021] + [2019, 2020] + [2019, 2020, 2021]
trees = [False] * len(alphas)
n_scenarios = [5] * 8 + [3] + [5] * 11
h_maxs = [-0.4] * 8 + [-0.3] * 12
hours = [48 if y!=2021 else 24 for y in years]
assert len(alphas) == len(cvars) == len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [12]:
# 13/11/23 1032
basename = 'optrun_dates_'
alphas = [0.99, 0.9, 0.9] + [0.9] * 2 + [0.99, 0.9, 0.8] + [0.9, 0.8] * 2
cvars = [-0.3, -0.395, -0.3] +[-0.3] * 2 + [-0.3] * 3 + [-0.3, -0.3] * 2
years = [2019] * 3 + [2019, 2020] +  [2019] * 3 + [2020] * 2 + [2021] * 2
trees = [False] * 3 + [False] * 2 + [False] * 3 + [False] * 4
n_scenarios = [5] * 3 + [3] * 2 + [5] * 3 + [5] * 4
h_maxs = [-0.4] * 3 + [-0.3] * 2 + [-0.3] * 3 + [-0.3] * 4
hours = [48] * 3 + [48] * 2 + [48] * 3 + [48] * 4
assert len(alphas) == len(cvars) == len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [1]:
# 15/11/2023 2257 FINISH CVAR EXPERIMENTS!
basename = 'optrun_dates_'
alphas = [0.99, 0.8]
cvars = [-0.3, -0.3]
years = [2019, 2019]
trees = [False, False]
n_scenarios = [5, 5]
h_maxs = [-0.3, -0.3]
hours = [48, 48]
assert len(alphas) == len(cvars) == len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [2]:
print(len(alphas))
print(len(years))
print(len(cvars))

2
2
2


In [7]:
i = 0
for alpha, cvar, year, tree, n, h_max, h in zip(alphas, cvars, years, trees, n_scenarios, h_maxs, hours):
    name = basename + str(i) + '.sh'
    make_slurm_job(savepath / name, h_max, n, -0.4, cvar, alpha, tree, year, h)
    i += 1

In [6]:
# 12/10/23 robust finish
basename = 'optrun_dates_'
i=0
h_maxs = [-0.4] * 2
trees = [False] * 2
ns = [5] * 2
years = [2019, 2020]

for h_max, tree, n, year in zip(h_maxs, trees, ns, years):
    name = basename + str(i) + '.sh'
    hour=24
    make_slurm_job_robust(savepath / name, h_max, n, -0.4, -0.4, 1, tree, year, hour)
    i+=1

In [7]:
# read the robust list
# entries will be (wl, shape, n, year)
with open('slurm_robust.pkl', 'rb') as f:
    robust_joblist = pickle.load(f)

# i=0
basename = 'optrun_dates'
for job in robust_joblist:
    h_max = job[0]
    tree = job[1]
    n = job[2]
    year = job[3]

    if year == 2021:
        hour = 24
    else:
        hour = 48

    name = basename + f'_{i}.sh'
    make_slurm_job_robust(savepath / name, h_max, n, -0.4, -0.4, 1, tree, year, hour)
    
    i += 1

In [4]:
# read the cvar list
# entries will be (wl, alpha, cvar, shape, n, year)
with open('slurm_cvars.pkl', 'rb') as f:
    cvar_joblist = pickle.load(f)

i=0 # comment out to continue from cell above
basename = 'optrun_dates'
for job in cvar_joblist:
    h_max = job[0]
    alpha = job[1]
    cvar = job[2]
    tree = job[3]
    n = job[4]
    year = job[5]

    if year == 2021:
        hours = 24
    else:
        hours = 48
    
    name = basename + f'_{i}.sh'
    make_slurm_job(savepath / name, h_max, n, -0.4, cvar, alpha, tree, year, hours)
    i += 1

In [9]:
# 29/09/23, finish n=3
basename = 'optrun_dates'

alphas = [0.9] * 6 + [0.8] * 3
cvars = [-0.3] * 9
years = [2019, 2020, 2021] * 3
trees = [False] * 9
n_scenarios = [3] * 9
h_maxs = [-0.4] * 3 + [-0.3] * 6
hours = [24] * 9

In [10]:

i=0

for h_max, alpha, cvar, year, tree, n, hour in zip(h_maxs, alphas, cvars, years, trees, n_scenarios, hours):
    name = f'{basename}_{i}.sh'
    make_slurm_job(savepath / name, h_max, n, -0.4, cvar, alpha, tree, year, hour)
    i+=1

In [93]:
i

72

In [46]:
years = [2019, 2020, 2021] * 3
trees = [True] * 3 + [False] * 3 + [True] * 3
n_scenarios = [3] * 9
h_maxs = [-0.4] * 3 + [-0.3] * 6
hours = [12] * 9

# check if all list are the same length
assert len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [8]:
years = [2019, 2020, 2021] * 4 # 260923
trees = [False] * 3 + [True] * 3 + [False] * 3 + [True] * 3
n_scenarios = [5] * 12
h_maxs = [-0.4] * 6 + [-0.3] * 6
hours = [24 if year<2021 else 12 for year in years]

# check if all list are the same length
assert len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [87]:
# Robust 11/09/23 10:34
years = [2020, 2019, 2020]
trees = [True] * 3
n_scenarios = [3] * 3
h_maxs = [-0.4] * 1 + [-0.3] * 2
hours = [12] * 3

# check if all list are the same length
assert len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [9]:
# Robust 15/09/23 12:14
years = [2019, 2020, 2021] * 4
trees = [False] * 3 + [True] * 3 + [False] * 3 + [True] * 3
n_scenarios = [3] * 12
h_maxs = [-0.4] * 6 + [-0.3] * 6
hours = [24 if year<2021 else 12 for year in years] 

# check if all list are the same length
assert len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [ ]:
# Robust 15/09/23 12:14
years = [2019, 2020, 2021] * 4
trees = [False] * 3 + [True] * 3 + [False] * 3 + [True] * 3
n_scenarios = [3] * 12
h_maxs = [-0.4] * 6 + [-0.3] * 6
hours = [24 if year<2021 else 12 for year in years] 

# check if all list are the same length
assert len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [12]:
# Robust 1 scen 15/09/23 12:19
years = [2019, 2020, 2021] * 2
trees = [False] * 6
n_scenarios = [1] * 6
h_maxs = [-0.4] * 3 + [-0.3] * 3
hours = [12 if year<2021 else 4 for year in years] 

# check if all list are the same length
assert len(years) == len(trees) == len(n_scenarios) == len(h_maxs) == len(hours)

In [9]:
basename = 'optrun_dates'

i=0
for h_max, year, tree, n, hour in zip(h_maxs, years, trees, n_scenarios, hours):
    name = f'{basename}_{i}.sh'
    make_slurm_job_robust(savepath / name, h_max, n, -0.4, -0.4, 1, tree, year, hour)
    i+=1

In [43]:
def make_slurm_job_obs(name, h_max, n, var, cvar, alpha, tree, year, hours):
    w = open(name, 'w', newline='\n')
    w.write('#!/bin/bash\n')
    w.write('#SBATCH -J optimization_run\n')
    w.write('#SBATCH --nodes=1\n')
    w.write('#SBATCH --ntasks=1\n')
    n_cpu = 32
    w.write(f'#SBATCH --cpus-per-task={n_cpu}\n') # can go to 128?
    # make the timestring from the hours
    timestring = str(int(hours)) + ':' + str(int((hours - int(hours))*60)) + ':00'
    w.write('#SBATCH --time=' + timestring + '\n')
    w.write('#SBATCH --partition=genoa\n')
    w.write('#SBATCH --output=slurm_%j.log\n')
    w.write('#SBATCH --error=slurm_%j.log\n')
    w.write("")
    w.write('module load 2022\n')
    w.write('module load Miniconda3/4.12.0\n')
    w.write('source /gpfs/admin/_hpc/sw/arch/AMD-ZEN2/RHEL8/EB_production/2022/software/Miniconda3/4.12.0/etc/profile.d/conda.sh\n')
    w.write('conda activate optimization\n')
    w.write("")
    w.write("method='obs'\n")
    w.write(f"var_wl={var}\n")
    w.write(f'max_threads={n_cpu}\n')
    w.write('start_from_scratch=False\n')
    w.write(f"p_chance=1\n")
    w.write("")
    w.write(f"wl_constraint_type='robust'\n")
    w.write('n_scenarios=' + str(n) + '\n')
    w.write(f"cvar_wl={cvar}\n")
    w.write(f"cvar_alpha={alpha}\n")
    w.write(f'tree_based={tree}\n')
    w.write(f'year={year}\n')
    w.write(f'h_max={h_max}\n')

    w.write(f'export OMP_NUM_THREADS=$max_threads\n')
    w.write('set -euo pipefail\n')

    w.write("python 'optimization/src/run_sim_date.py' n_scenarios=$n_scenarios distance_metric=$method wl_constraint_type=$wl_constraint_type var_wl=$var_wl cvar_wl=$cvar_wl cvar_alpha=$cvar_alpha p_chance=$p_chance tree_based=$tree_based max_threads=$max_threads start_from_scratch=$start_from_scratch h_max=$h_max year=$year")


In [49]:
basename = 'optrun_dates' # 09/09/23 16:26
for i, year in enumerate([2019, 2020, 2021]):
    name = f'{basename}_{i}.sh'
    if year == 2021:
        h = 1
    else:
        h = 1
    make_slurm_job_obs(savepath / name, -0.4, 1, -0.4, -0.4, 1, False, year, h)